In [ ]:
import bagpy
import numpy as np
import matplotlib.pyplot as plt
import rosbag

In [ ]:
from frenet_converter import frenet_converter

In [ ]:
bag_name = "/home/gnone/rosbags/frenet_bug.bag"

boundaries_topic_name = "/trackbounds/markers"
laserscan_topic_name = "/scan"
carstate_pose_topic_name = "/car_state/pose"
opponent_topic = "/perception/obstacles"
raceline_topic = "/global_waypoints"
map_topic_name = "/map"

bag = bagpy.bagreader(bag_name)

In [ ]:
# proper IEEE sizes 

# 1 column
COLWIDTH = 3.5 # in inches
FONTSIZE = 8 # in pt

In [ ]:
bag_obj = rosbag.Bag(bag_name)

In [ ]:
#get the raceline
raceline = None
for topic, msg, t in bag_obj.read_messages(topics=[raceline_topic]):
    raceline = msg.wpnts
    break

raceline_x = np.array([p.x_m for p in raceline])
raceline_y = np.array([p.y_m for p in raceline])

#Frenet converter
fc = frenet_converter.FrenetConverter(raceline_x, raceline_y)

In [ ]:
x = 10.281317029280368
y = 18.758460617419132

x1 = 8.550580978393555
y1 = 14.897018432617188

x2 = 8.650595664978027
y2 = 15.354464530944824

resp = fc.get_frenet([x, x1, x2], [y, y1, y2])
print(resp)

In [ ]:
# print(resp[0], resp[1])
cart = fc.get_cartesian(resp[0], resp[1])
print([f"Reprojection error: {num:.3f} [m]" for num in (np.linalg.norm(cart, axis=0) - np.linalg.norm([[x, x1, x2], [y, y1, y2]], axis=0))])

In [ ]:
spline_x, spline_y = fc.get_splines()

In [ ]:
space = np.linspace(20, 21, 2000)
# set aspect equal 
fig, ax = plt.subplots(figsize=(COLWIDTH, COLWIDTH))
ax.set_aspect('equal')
ax.plot(spline_x(space), spline_y(space))

ax.plot(x,y,'x')
# print tangent 
for i in np.linspace(20.6, 20.59, 3):
    tangent = [spline_x(i, 1), spline_y(i, 1)]
    # tangent = [-tangent[1], tangent[0]]
    # normalize 
    tangent = tangent / np.linalg.norm(tangent, axis=0)
    # print(tangent)
    second_guy = np.array([x-spline_x(i), y-spline_y(i)])
    # print(second_guy)
    proj = np.clip(np.dot(tangent, second_guy), -1.66666667e-02, 1.66666667e-02)
    print(proj)
    print(np.einsum('ij,ij->i', [tangent], [second_guy])-proj)

ax.plot(spline_x([20.5, 20.6]), spline_y([20.5, 20.6]), 'o')
ax.plot([spline_x(20.6), spline_x(20.6) + tangent[0]], [spline_y(20.6), spline_y(20.6) + tangent[1]], 'r')